In [ ]:
!git add .
!git commit -m "Merged old news with EIA"
!git push

In [2]:
from data.data_functions import *
import math
import datetime as dt
import sqlite3
import pandas as pd

from datashop import *

path = 'c:/prompt_root/CrudeOilApp'
relpath = '/data/nyt_api/nyt_jsons/'

conn = sqlite3.connect('data/energydash.db')

#test_bulk = jsons_to_frame(path,relpath,conn)
#test_bulk.to_csv('data/nyt_api/nyt_articles.csv')

# EIA Data

In [3]:
# EIA_Bulk

eia_dict = {
    'PET.WTTSTUS1.W':['WeeklyStocks','235081','%Y%m%d'],
    'PET.RWTC.D':['DailyPrice','241335','%Y%m%d'],
    'PET.WRPUPUS2.W':['ProductSupplied','401676','%Y%m%d'],
    'PET.E_ERTRRO_XR0_NUS_C.M':['MonthlyRigCount','296749','%Y%m']
}

for key,val in eia_dict.items():

    e_ser = EIA_Series(key,name = val[0],date_format = val[2], end =            '20200910')

    e_ser.frame.to_sql(val[0],conn,if_exists='replace')  

    query = " SELECT * FROM {} ORDER BY Date DESC LIMIT 5".format(val[0])
    df = pd.read_sql(query,conn)
    print(df)

                  Date  WeeklyStocks   date_only
0  2020-09-04 00:00:00       2072722  2020-09-04
1  2020-08-28 00:00:00       2076448  2020-08-28
2  2020-08-21 00:00:00       2085502  2020-08-21
3  2020-08-14 00:00:00       2095037  2020-08-14
4  2020-08-07 00:00:00       2100319  2020-08-07
                  Date  DailyPrice   date_only
0  2020-09-10 00:00:00       37.25  2020-09-10
1  2020-09-09 00:00:00       38.05  2020-09-09
2  2020-09-08 00:00:00       36.87  2020-09-08
3  2020-09-04 00:00:00       39.69  2020-09-04
4  2020-09-03 00:00:00       41.39  2020-09-03
                  Date  ProductSupplied   date_only
0  2020-09-04 00:00:00            18678  2020-09-04
1  2020-08-28 00:00:00            16979  2020-08-28
2  2020-08-21 00:00:00            19619  2020-08-21
3  2020-08-14 00:00:00            17159  2020-08-14
4  2020-08-07 00:00:00            19369  2020-08-07
                  Date  MonthlyRigCount   date_only
0  2020-05-01 00:00:00              267  2020-05-01
1  2020-

In [4]:
eia_dict = {
    'PET.WTTSTUS1.W':['WeeklyStocks','235081','%Y%m%d'],
    'PET.RWTC.D':['DailyPrice','241335','%Y%m%d'],
    'PET.WRPUPUS2.W':['ProductSupplied','401676','%Y%m%d'],
    'PET.E_ERTRRO_XR0_NUS_C.M':['MonthlyRigCount','296749','%Y%m']
}

with open ('data/cache.json','r') as cache_file:
    cache_dict = json.load(cache_file)

#_________ Base URL. Requires category ID to return series in that category"

eia_api_url= 'http://api.eia.gov/updates/?api_key=651b30b69f4f47a13a2912d673f7da93&category_id='



# ________ Get last update time for each series. 

current_eia = {}

for key,val in eia_dict.items():

    url = eia_api_url + val[1] + '&rows=10000'
    request = requests.get(url)
    dict = json.loads(request.text)

    # Load last updated time from DB
    
    query = " SELECT MAX(Date) from {}".format(val[0])
    last_date = conn.execute(query).fetchall()[0][0]
    last_date_eia = dt.datetime.strptime(last_date[:10], '%Y-%m-%d').strftime(val[2])

    last_update = 'not found'
    #Get last updated time from API
    for update in dict['updates']:
        if update['series_id'] == key:
            last_update = update['updated']
            last_update = dt.datetime.strptime(last_update[:10], '%Y-%m-%d').strftime(val[2])

    #compare both, get updates if needed and add to table

    if last_update > last_date:
        new_series = EIA_Series(
            key, 
            name=val[0],
            start = last_date_eia,
            date_format = val[2]).frame

        new_series = new_series[new_series.index > pd.Timestamp(last_date)]
        
        new_series.to_sql(val[0],conn,if_exists='append')   

    # get the current last value for later use

    query = " SELECT * FROM  {} WHERE Date = (SELECT MAX(Date) FROM {})".format(val[0],val[0])
    last_date = conn.execute(query).fetchall()[0]

    current_eia[val[0]] = last_date

print(current_eia)


{'WeeklyStocks': ('2020-10-30 00:00:00', 2015817, '2020-10-30'), 'DailyPrice': ('2020-11-03 00:00:00', 37.44, '2020-11-03'), 'ProductSupplied': ('2020-10-30 00:00:00', 18362, '2020-10-30'), 'MonthlyRigCount': ('2020-05-01 00:00:00', 267, '2020-05-01')}


# News Data 

In [5]:
frame = jsons_to_frame(path,relpath,conn)

 'news' table replaced in energydash.db


In [7]:
query = " SELECT * FROM news ORDER BY Date DESC LIMIT 10"
df = pd.read_sql(query,conn)
print(df['date_only'])

0    2020-10-09
1    2020-10-07
2    2020-10-01
3    2020-09-28
4    2020-09-24
5    2020-09-23
6    2020-09-21
7    2020-09-17
8    2020-09-15
9    2020-09-12
Name: date_only, dtype: object


In [10]:
start_date = '2020-02-23 17:00:07'	
end_date = str(dt.datetime.now())

quer = 'Oil (Petroleum) and Gasoline'

test_call = nytResp(start_date,end_date,quer)

if test_call.hits > 0:
    conn = sqlite3.connect('data/energydash.db')
    test_call.frame.to_sql('news',conn,if_exists='append')



In [13]:
query = ''' 
    SELECT * 
    FROM news 
    LEFT JOIN DailyPrice ON news.date_only = DailyPrice.date_only
    ORDER BY Date

'''
df = pd.read_sql(query,conn)
display(df.tail(30)[['abstract','date_only','Date','DailyPrice']])

,abstract,date_only,date_only,Date,Date,DailyPrice
12632,"A giant new vessel, OceanXplorer, seeks to unv...",2020-09-17,2020-09-17,2020-09-17 09:00:34,2020-09-17 00:00:00,40.99
12633,"A giant new vessel, OceanXplorer, seeks to unv...",2020-09-17,2020-09-17,2020-09-17 09:00:34,2020-09-17 00:00:00,40.99
12634,While BP and other European companies invest b...,2020-09-21,2020-09-21,2020-09-21 09:00:25,2020-09-21 00:00:00,39.26
12635,While BP and other European companies invest b...,2020-09-21,2020-09-21,2020-09-21 09:00:25,2020-09-21 00:00:00,39.26
12636,The proposal would speed up the state’s effort...,2020-09-23,2020-09-23,2020-09-23 19:12:59,2020-09-23 00:00:00,39.92
12637,The proposal would speed up the state’s effort...,2020-09-23,2020-09-23,2020-09-23 19:12:59,2020-09-23 00:00:00,39.92
12638,This documentary follows individuals pushing b...,2020-09-24,2020-09-24,2020-09-24 11:00:09,2020-09-24 00:00:00,40.11
12639,This documentary follows individuals pushing b...,2020-09-24,2020-09-24,2020-09-24 11:00:09,2020-09-24 00:00:00,40.11
12640,"In a conversation with Kara Swisher, the billi...",2020-09-28,2020-09-28,2020-09-28 09:00:05,2020-09-28 00:00:00,40.47
12641,"In a conversation with Kara Swisher, the billi...",2020-09-28,2020-09-28,2020-09-28 09:00:05,2020-09-28 00:00:00,40.47


In [ ]:
query = ''' 
    SELECT * 
    FROM DailyPrice 

'''
df = pd.read_sql(query,conn)
display(df.tail())

In [ ]:
# Testing News

conn = sqlite3.connect('data/energydash.db')

query = 'SELECT date,abstract,url,retrieved FROM news ORDER BY date DESC LIMIT 10'
df = pd.read_sql_query(query,conn)